In [39]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.layers import TimeDistributed, MaxPooling2D, Dropout, LSTM, ConvLSTM2D
from keras.applications.xception import Xception
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import sklearn.metrics as metrics

import numpy as np
import pandas as pd

import pickle
import sys, gc, os
import glob

import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
d=128

In [3]:
# https://keras.io/examples/cifar10_resnet/
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


In [29]:
le = LabelEncoder()

train = np.empty(shape=(0,16,d,d,3), dtype=np.int8)
labels = np.empty(shape=(0,2), dtype=np.float)

for f in glob.glob("../data/train_lstm1_*"):
    print(f)
    train_loader = np.load(f)
    fakes = train_loader['fakes']
    train = np.append(train,fakes,axis=0)
    print('fakes',len(fakes))
    reals = train_loader['reals']
    train = np.append(train,reals,axis=0)
    print('reals',len(reals))
    l = np.append(np.full(len(fakes),'FAKE'),np.full(len(reals),'REAL'))
    l = le.fit_transform(l)
    l = np_utils.to_categorical(l, 2)
    print('labels',labels.ndim,l.ndim)
    labels = np.append(labels,l,axis=0)
    
print('train=',len(train))
print('labels',len(labels))

../data/train_lstm1_15.npz
fakes 27
reals 27
labels 2 2
../data/train_lstm1_4.npz
fakes 25
reals 26
labels 2 2
../data/train_lstm1_5.npz
fakes 28
reals 26
labels 2 2
../data/train_lstm1_14.npz
fakes 24
reals 25
labels 2 2
../data/train_lstm1_16.npz
fakes 23
reals 26
labels 2 2
../data/train_lstm1_17.npz
fakes 26
reals 27
labels 2 2
../data/train_lstm1_13.npz
fakes 28
reals 25
labels 2 2
../data/train_lstm1_2.npz
fakes 24
reals 24
labels 2 2
../data/train_lstm1_3.npz
fakes 25
reals 28
labels 2 2
../data/train_lstm1_12.npz
fakes 28
reals 29
labels 2 2
../data/train_lstm1_1.npz
fakes 23
reals 21
labels 2 2
../data/train_lstm1_11.npz
fakes 27
reals 27
labels 2 2
../data/train_lstm1_19.npz
fakes 27
reals 27
labels 2 2
../data/train_lstm1_18.npz
fakes 28
reals 28
labels 2 2
train= 729
labels 729


In [5]:
# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 200
num_classes=2

# Input image dimensions.
input_shape = train[1].shape
print(input_shape)


(16, 128, 128, 3)


In [46]:
def lstm1(num_frames, width, height, num_classes):
    seq = Sequential()
    seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(None, width, height, 3),
                   padding='same', return_sequences=True))
    seq.add(BatchNormalization())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

In [35]:
def lstm1(num_frames, width, height, num_classes):
    model = Sequential()
    model.add(Input(shape=(num_frames, width, height, 3)))
    model.add(LSTM(2048, 
                   return_sequences=False,
                   dropout=0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    return model

In [47]:
model = lstm1(num_frames=16, width=d, height=d, num_classes=2)


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
model.summary()

# Prepare model model saving directory.
save_dir = '../models'
model_name = 'DDC_LSTM1_model.{epoch:03d}.h5' 
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=0.2,
                               cooldown=0,
                               patience=2,
                               min_lr=0.5e-6)

earlystop = EarlyStopping(monitor='val_loss',
                          patience=6)

callbacks = [checkpoint, lr_reducer, lr_scheduler, earlystop]



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 2048)              16809984  
_________________________________________________________________
dense_5 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 1026      
_________________________________________________________________
dense_7 (Dense)              (None, 512)               1536      
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 1026      
__________

In [30]:
(trainX, testX, trainY, testY) = train_test_split(train, labels, test_size=0.25, random_state=42)

In [31]:
del train, labels

In [48]:
history = model.fit(trainX,
              trainY,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(testX, testY),
              shuffle=True,
              callbacks=callbacks)

ValueError: Error when checking input: expected lstm_9_input to have 3 dimensions, but got array with shape (546, 16, 128, 128, 3)

In [ ]:
f, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2, figsize=(15, 8))

# Plot training & validation accuracy values
ax1.plot(history.history['acc'])
ax1.plot(history.history['val_acc'])
ax1.set_title('Model accuracy')
ax1.set(xlabel="Epoch", ylabel="Accuracy")
ax1.legend(['Train', 'Test'], loc='down right')

# Plot training & validation loss values
ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_title('Model loss')
ax2.set(xlabel="Epoch", ylabel="Loss")
ax2.legend(['Train', 'Test'], loc='upper right')

plt.show()

In [ ]:
predictions = model.predict(faces)

In [ ]:
metrics.log_loss(labels[:,0],predictions[:,0])